

### <span style="color:purple; font-weight:bold;">Project Name: Netflix Content Strategy Analysis</span>
### <span style="color:purple; font-weight:bold;">Person Name: Anirban Bose</span>
<h3><strong style="color: purple;">Data Source: </strong><a href="https://statso.io/netflix-content-strategy-case-study/" target="_blank">Netflix Content Strategy Case Study</a></h3>


#### **I. Import the modules and packages**

In [168]:
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go 
import plotly.io as pio
from IPython.display import display, HTML

In [169]:
# Set the default template to Dark theme
pio.templates.default = "plotly_white"

#### **II. Declare the global variables, dictionaries etc.**

In [170]:
# Map month numbers to month names
month_map = {
    1: "January", 2: "February", 3: "March", 4: "April",
    5: "May", 6: "June", 7: "July", 8: "August",
    9: "September", 10: "October", 11: "November", 12: "December"
}

#### **III. Functions**

In [171]:
# Function to convert column names to snakecase

def func_convert_to_snake_case(df):
    """
    Converts column names of a pandas DataFrame to snake_case.
    
    Parameters:
        df (pd.DataFrame): The DataFrame whose columns need to be converted.
        
    Returns:
        pd.DataFrame: The DataFrame with updated column names in snake_case.
    """
    df.columns = (
        df.columns.str.strip()  # Remove leading/trailing spaces
        .str.lower()  # Convert to lowercase
        .str.replace(r"[ ()-]", "_", regex=True)  # Replace spaces, parentheses, and hyphens with underscores
        .str.replace(r"__+", "_", regex=True)  # Replace multiple underscores with a single underscore
        .str.rstrip("_")  # Remove trailing underscores
        .str.replace("?", "", regex=False) # Remove question mark from column name
    )
    return df

In [172]:
# Function to convert datatypes
def func_change_dtypes(df):
    """
    Cleans and converts specific columns of a DataFrame:
    - Converts 'available_globally' to boolean.
    - Converts 'release_date' to datetime.
    - Converts 'hours_viewed' to numeric.
    
    Parameters:
        df (pd.DataFrame): The input DataFrame to clean and transform.
    
    Returns:
        pd.DataFrame: The cleaned and transformed DataFrame.
    """
    # Step 1: Convert 'available_globally' to boolean
    df["available_globally"] = df["available_globally"].map({"Yes": True, "No": False})
    
    # Step 2: Convert 'release_date' to datetime
    df["release_date"] = pd.to_datetime(df["release_date"], format="%Y-%m-%d", errors="coerce")
    
    # Step 3: Convert 'hours_viewed' to numeric
    df["hours_viewed"] = (
            df["hours_viewed"]
            .str.replace(",", "", regex=False)  # Remove commas
            .astype(int)  # Convert to integer
        )
    
    return df

In [173]:
# Function for feature engineering

def func_feature_engineering_preEDA(df, date_col):

    # Extract the month from the date column
    df["release_month"] = df[date_col].dt.month

    df["release_month_name"] = df["release_month"].map(month_map)    

    # Extract the day name from the date column
    df["release_day_name"] = df[date_col].dt.day_name()

    return df

In [174]:
# Common function for bar chart
def create_bar_chart(df, group_col, title, x_label, y_label, group_category):
    """
    Generates a bar chart for `hours_viewed` grouped by the specified column.
    
    Parameters:
        df (pd.DataFrame): The input DataFrame.
        group_col (str): The column name to group by.
        title (str): The title of the chart.
        x_label (str): Label for the x-axis.
        y_label (str): Label for the y-axis.
    
    Returns:
        None: Displays the chart.
    """
     # Group and sort data
    if group_category == "sum":
        grouped_data = (
            df[[group_col, 'hours_viewed']]
            .groupby(group_col, as_index=False)
            .sum()
            .sort_values(by="hours_viewed", ascending=False)  # Sort in descending order
        )
    elif (group_category == "mean" or group_category == "avg"):
        grouped_data = (
            df[[group_col, 'hours_viewed']]
            .groupby(group_col, as_index=False)
            .mean()
            .sort_values(by="hours_viewed", ascending=False)  # Sort in descending order
        )

    fig = px.bar(
        grouped_data,
        x=group_col,
        y="hours_viewed",
        title=title,
        labels={group_col: x_label, "hours_viewed": y_label},
        color=group_col
    )
    fig.show()

In [175]:
# Function to create plot trends in viewership data by release month or day name.

def func_plot_trend_viewership(df, trend_type, viewership_col, split_column, title, group_category):
    
    # Ensure valid trend_type
    if trend_type not in ["month", "day"]:
        raise ValueError("trend_type must be either 'month' or 'day'.")

    # Select grouping column based on trend_type
    if trend_type == "month":
        group_col = "release_month_name"  # Use release_month_name for better readability
        month_order = [
            "January", "February", "March", "April", "May", "June", 
            "July", "August", "September", "October", "November", "December"
        ]
        df[group_col] = pd.Categorical(df[group_col], categories=month_order, ordered=True)
        tickvals = month_order
    elif trend_type == "day":
        group_col = "release_day_name"  # Use release_day_name for day analysis
        day_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
        df[group_col] = pd.Categorical(df[group_col], categories=day_order, ordered=True)
        tickvals = day_order

    # Aggregate viewership data
    if split_column == "" and group_category == 'sum':
        trend_viewership = df.groupby(group_col)[viewership_col].sum().reset_index()
    elif split_column == "" and group_category == 'mean':
        trend_viewership = df.groupby(group_col)[viewership_col].mean().reset_index()    
    elif split_column != "" and group_category == 'sum':
        trend_viewership = df.groupby([group_col, split_column])[viewership_col].sum().reset_index()       
    elif split_column != "" and group_category == 'mean':
        trend_viewership = df.groupby([group_col, split_column])[viewership_col].mean().reset_index()

    # Create traces for plotting
    traces = []
    if split_column == "":
        # Single trace if no split column is provided
        traces.append(
            go.Scatter(
                x=trend_viewership[group_col],
                y=trend_viewership[viewership_col],
                mode="lines+markers",
                name="Total Viewership",
                marker=dict(size=8),
                line=dict(width=2),
            )
        )
    else:
        # Traces for each unique value in split_column
        for split_column_value in trend_viewership[split_column].unique():
            content_data = trend_viewership[trend_viewership[split_column] == split_column_value]
            traces.append(
                go.Scatter(
                    x=content_data[group_col],
                    y=content_data[viewership_col],
                    mode="lines+markers",
                    name=str(split_column_value),
                    marker=dict(size=8),
                    line=dict(width=2),
                )
            )

    # Create the figure
    fig = go.Figure(data=traces)

    # Update layout with custom title, axis labels, and ticks
    fig.update_layout(
        title=title,
        xaxis_title="Month" if trend_type == "month" else "Day of the Week",
        yaxis_title="Total Hours Viewed",
        xaxis=dict(
            tickmode="array",
            tickvals=tickvals,
            ticktext=tickvals,
        ),
        legend_title=split_column if split_column else "Viewership",
        height=600,
        width=1000,
    )

    # Show the plot
    fig.show()

In [206]:
# Function to create a bar + line chart to show the number of releases and viewer hours.
def func_combined_chart(
    df, trend_type, release_col, viewership_col, title
):
    """
    Create a bar + line chart to show the number of releases and viewer hours.

    Parameters:
        df (pd.DataFrame): The input dataframe.
        trend_type (str): 'month' or 'day' to specify trend type.
        release_col (str): The column indicating the release date or month/day.
        viewership_col (str): The column containing viewer hours.
        title (str): The title of the plot.
    """
    # Ensure valid trend_type
    if trend_type not in ["month", "day"]:
        raise ValueError("trend_type must be either 'month' or 'day'.")

    # Select grouping column and ensure ordering
    if trend_type == "month":
        group_col = "release_month_name"  # Use release_month_name for months
        month_order = [
            "January", "February", "March", "April", "May", "June", 
            "July", "August", "September", "October", "November", "December"
        ]
        df[group_col] = pd.Categorical(df[group_col], categories=month_order, ordered=True)
        tickvals = month_order
    elif trend_type == "day":
        group_col = "release_day_name"  # Use release_day_name for days
        day_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
        df[group_col] = pd.Categorical(df[group_col], categories=day_order, ordered=True)
        tickvals = day_order

    # Filter rows where the grouping column is not null
    df = df[df[group_col].notna()]

    # Aggregate data for releases (count) and viewership (sum)
    releases_count = df.groupby(group_col)[release_col].count().reset_index(name="releases")
    viewer_hours = df.groupby(group_col)[viewership_col].sum().reset_index(name="viewer_hours")

    # Merge the two datasets to align data
    combined_data = pd.merge(releases_count, viewer_hours, on=group_col)

    # Create traces for bar and line charts
    bar_trace = go.Bar(
        x=combined_data[group_col],
        y=combined_data["releases"],
        name="Number of Releases",
        marker=dict(color='rgb(221, 160, 221)'),  # Light blue color
        yaxis="y1"  # Explicitly tie this trace to the primary y-axis
    )

    line_trace = go.Scatter(
        x=combined_data[group_col],
        y=combined_data["viewer_hours"],
        name="Viewer Hours",
        mode="lines+markers",
        line=dict(color='rgb(26, 118, 255)', width=2),
        yaxis="y2"  # Explicitly tie this trace to the secondary y-axis
    )

    # Create the figure with dual y-axes
    fig = go.Figure(data=[bar_trace, line_trace])

    # Update layout with dual y-axes
    fig.update_layout(
        title=title,
        xaxis=dict(
            title="Month" if trend_type == "month" else "Day of the Week",
            tickmode="array",
            tickvals=tickvals,
            ticktext=tickvals
        ),
        yaxis=dict(
            title="Number of Releases",
            showgrid=False,
            side="left"  # Primary y-axis on the left
        ),
        yaxis2=dict(
            title="Viewer Hours",
            overlaying="y",  # Overlay on the same plot
            side="right",  # Secondary y-axis on the right
            showgrid=False
        ),
        legend=dict(
            x=0.1,
            y=1.1,
            orientation="h"
        ),
        height=600,
        width=1000,
    )

    # Show the plot
    fig.show()


In [176]:
# Function to create scrollable table within a small window
def create_scrollable_table(df, table_id, title):
    html = f'<h3>{title}</h3>'
    html += f'<div id="{table_id}" style="height:200px; overflow:auto;">'
    html += df.to_html()
    html += '</div>'
    return html

#### **IV. Exploratory Data Analysis**

##### <span style="color:blue; font-weight:bold;">Read the input file</span>

In [177]:
# Go back to parent directory
parent_dir = os.path.dirname(os.getcwd())

# Create the filepath
filename = 'netflix_content_2023.csv'
filename_folder = 'input'
filepath = os.path.join(parent_dir, filename_folder, filename)

# Read the csv file and create the dataframe
df_netflix_content = pd.read_csv(filepath)

In [178]:
# Get the number of rows and columns
df_netflix_content.shape

(24812, 6)

##### <span style="color:blue; font-weight:bold;">Basic statistical analysis and data cleaning</span>

In [179]:
df_netflix_content.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24812 entries, 0 to 24811
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Title                24812 non-null  object
 1   Available Globally?  24812 non-null  object
 2   Release Date         8166 non-null   object
 3   Hours Viewed         24812 non-null  object
 4   Language Indicator   24812 non-null  object
 5   Content Type         24812 non-null  object
dtypes: object(6)
memory usage: 1.1+ MB


<strong><u>Insights:</u></strong>

We realize here that the the columns are not in standard format and we need to convert them to snakecase

In [180]:
# convert function to snakecase
df_netflix_content = func_convert_to_snake_case(df_netflix_content)
df_netflix_content.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24812 entries, 0 to 24811
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   title               24812 non-null  object
 1   available_globally  24812 non-null  object
 2   release_date        8166 non-null   object
 3   hours_viewed        24812 non-null  object
 4   language_indicator  24812 non-null  object
 5   content_type        24812 non-null  object
dtypes: object(6)
memory usage: 1.1+ MB


In [181]:
# check if available_globally column is a binary. If yes, then this column can be converted to boolean
for col in df_netflix_content.columns:
    unique_values = df_netflix_content[col].unique()
    print(f"{col}: {unique_values}")

title: ['The Night Agent: Season 1' 'Ginny & Georgia: Season 2'
 'The Glory: Season 1 // 더 글로리: 시즌 1' ... 'Terror y Feria: Season 1'
 'The Witcher: A Look Inside the Episodes: Limited Series'
 'We Are Black and British: Season 1']
available_globally: ['Yes' 'No']
release_date: ['2023-03-23' '2023-01-05' '2022-12-30' ... '2023-12-04' '2023-12-21'
 '2015-12-04']
hours_viewed: ['81,21,00,000' '66,51,00,000' '62,28,00,000' '50,77,00,000'
 '50,30,00,000' '44,06,00,000' '42,96,00,000' '40,25,00,000'
 '30,21,00,000' '26,62,00,000' '26,26,00,000' '25,25,00,000'
 '25,15,00,000' '24,99,00,000' '23,50,00,000' '23,48,00,000'
 '22,97,00,000' '22,11,00,000' '21,41,00,000' '20,97,00,000'
 '20,65,00,000' '20,55,00,000' '20,18,00,000' '20,07,00,000'
 '19,47,00,000' '19,29,00,000' '18,40,00,000' '18,23,00,000'
 '18,18,00,000' '17,68,00,000' '17,55,00,000' '17,43,00,000'
 '17,36,00,000' '17,24,00,000' '17,01,00,000' '16,83,00,000'
 '16,30,00,000' '16,20,00,000' '16,11,00,000' '15,76,00,000'
 '15,53,00,00

<strong><u>Insights:</u></strong>

We realize that the datatypes of the following columns need to be change:
1. available_globally: from string to boolean
2. release_date: from string to date
3. hours_viewed: from string to integer

In [182]:
# Change data types
df_netflix_content = func_change_dtypes(df_netflix_content)
df_netflix_content.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24812 entries, 0 to 24811
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   title               24812 non-null  object        
 1   available_globally  24812 non-null  bool          
 2   release_date        8166 non-null   datetime64[ns]
 3   hours_viewed        24812 non-null  int32         
 4   language_indicator  24812 non-null  object        
 5   content_type        24812 non-null  object        
dtypes: bool(1), datetime64[ns](1), int32(1), object(3)
memory usage: 896.7+ KB


##### <span style="color:blue; font-weight:bold;">Feature Engineering before EDA</span>

In [183]:
#1. We create a column release_month to analyze monthly trends
#2. We create a column release_day_name to analyze day_name trends 

df_netflix_content = func_feature_engineering_preEDA(df_netflix_content, 'release_date')
df_netflix_content.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24812 entries, 0 to 24811
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   title               24812 non-null  object        
 1   available_globally  24812 non-null  bool          
 2   release_date        8166 non-null   datetime64[ns]
 3   hours_viewed        24812 non-null  int32         
 4   language_indicator  24812 non-null  object        
 5   content_type        24812 non-null  object        
 6   release_month       8166 non-null   float64       
 7   release_month_name  8166 non-null   object        
 8   release_day_name    8166 non-null   object        
dtypes: bool(1), datetime64[ns](1), float64(1), int32(1), object(5)
memory usage: 1.4+ MB


In [184]:
df_netflix_content.head(5)

,title,available_globally,release_date,hours_viewed,language_indicator,content_type,release_month,release_month_name,release_day_name
0,The Night Agent: Season 1,True,2023-03-23,812100000,English,Show,3.0,March,Thursday
1,Ginny & Georgia: Season 2,True,2023-01-05,665100000,English,Show,1.0,January,Thursday
2,The Glory: Season 1 // 더 글로리: 시즌 1,True,2022-12-30,622800000,Korean,Show,12.0,December,Friday
3,Wednesday: Season 1,True,2022-11-23,507700000,English,Show,11.0,November,Wednesday
4,Queen Charlotte: A Bridgerton Story,True,2023-05-04,503000000,English,Movie,5.0,May,Thursday


##### <span style="color:blue; font-weight:bold;">Exploratory Data Analysis (EDA)</span>

In [185]:
# List of configurations for the bar charts
chart_configs = [
    {
        "group_col": "available_globally",
        "title": "Total Hours Viewed by Available Globally",
        "x_label": "Available Globally",
        "y_label": "Total Hours Viewed",
        "group_category": "sum"
    },
    {
        "group_col": "available_globally",
        "title": "Average Hours Viewed by Available Globally",
        "x_label": "Available Globally",
        "y_label": "Average Hours Viewed",
        "group_category": "mean"
    },
    {
        "group_col": "language_indicator",
        "title": "Total Hours Viewed by Language Indicator",
        "x_label": "Language Indicator",
        "y_label": "Total Hours Viewed",
        "group_category": "sum"
    },
    {
        "group_col": "language_indicator",
        "title": "Average Hours Viewed by Language Indicator",
        "x_label": "Language Indicator",
        "y_label": "Average Hours Viewed",
        "group_category": "mean"
    },
    {
        "group_col": "content_type",
        "title": "Total Hours Viewed by Content Type",
        "x_label": "Content Type",
        "y_label": "Total Hours Viewed",
        "group_category": "sum"
    },
    {
        "group_col": "content_type",
        "title": "Average Hours Viewed by Content Type",
        "x_label": "Content Type",
        "y_label": "Average Hours Viewed",
        "group_category": "mean"
    }
]

# Loop through the configurations and create bar charts
for config in chart_configs:
    create_bar_chart(
        df=df_netflix_content,
        group_col=config["group_col"],
        title=config["title"],
        x_label=config["x_label"],
        y_label=config["y_label"],
        group_category=config["group_category"]
    )

<strong><u>Insights:</u></strong>

1. Clearly, global shows are more popular and have more hours of viewership.
2. Curiously, even though English language has more hours of viewership, Korean language has more average viewership
3. Overall, shows are more popular than movies.

In [186]:
# Check for monthly trends

func_plot_trend_viewership(
    df=df_netflix_content,
    trend_type= 'month',
    viewership_col='hours_viewed',
    split_column = "",
    title='Total Viewership Hours by Release Month (2023)',
    group_category = "sum"

)

<strong><u>Insights:</u></strong>

June and Dec seems to have noticiable spikes. Is might be because of the summer and winter holidays that people have more time to watch shows.

Let us see what they are watching each month

In [188]:
# Check for monthly trends by content type

func_plot_trend_viewership(
    df=df_netflix_content,
    trend_type= 'month',
    viewership_col='hours_viewed',
    split_column = 'content_type',
    title='Total Viewership Hours by Release Month (2023) for different content types',
    group_category = 'sum')

<strong><u>Insights:</u></strong>

Shows follow the overall trend, while movies do not.

Let us see what language viewers are watching each month

In [190]:
# Check for monthly trends by language

func_plot_trend_viewership(
    df=df_netflix_content,
    trend_type= 'month',
    viewership_col='hours_viewed',
    split_column = 'language_indicator',
    title='Total Viewership Hours by Release Month (2023) for different languages',
    group_category = 'sum')

<strong><u>Insights:</u></strong>

Though English language has spikes in the last quarter of the year probably because of the holiday seasons, the spike is Korean content during June and December are noticeable and might be contributing to the overall viewing trend. 

In [191]:
# Check for day of week release trends 

func_plot_trend_viewership(
    df=df_netflix_content,
    trend_type= 'day',
    viewership_col='hours_viewed',
    split_column = '',
    title='Total Viewership Hours by Release day (2023)',
    group_category = 'sum')

<strong><u>Insights:</u></strong>

The release day spike in Friday corresponds to rise in viewership hours. Now, let's check whether this spike is for any specific language. 

In [193]:
# Check for day of week release trends by language indicator

func_plot_trend_viewership(
    df=df_netflix_content,
    trend_type= 'day',
    viewership_col='hours_viewed',
    split_column = 'language_indicator',
    title='Total Viewership Hours by Release day (2023) for different languages',
    group_category = 'sum')


func_plot_trend_viewership(
    df=df_netflix_content,
    trend_type= 'day',
    viewership_col='hours_viewed',
    split_column = 'language_indicator',
    title='Average Viewership Hours by Release day (2023) for different languages',
    group_category = 'mean')

<strong><u>Insights:</u></strong>

The Korean and Non-English language contents also follows this spike but not as pronounced as the spike for English language i.e. English language has the highest spike on Fridays. The average hours viewed is highest on Saturdays for Korean language contents.

In [204]:
func_combined_chart(
    df=df_netflix_content,
    trend_type="month",
    release_col="title",  # Count of releases
    viewership_col="hours_viewed",  # Viewer hours
    title="Number of Releases and Viewer Hours by Month"
)

<strong><u>Insights:</u></strong>

There is no correlation between number of releases in a month and the viewer hours. It seems like more than content quality, people's viewing choices are driven by personal pereferences (such as leisure time etc)

In [205]:
func_combined_chart(
    df=df_netflix_content,
    trend_type="day",
    release_col="title",  # Count of releases
    viewership_col="hours_viewed",  # Viewer hours
    title="Number of Releases and Viewer Hours by Day"
)


<strong><u>Insights:</u></strong>

There is clear correlation between number of releases in a day and the viewer hours. Friday seems to be the best day for releasing content.

##### <span style="color:blue; font-weight:bold;">Content Release Strategy for Netflix</span>

**Content Strategy for Netflix**

Based on the insights derived from the Netflix content analysis, the following strategies are recommended:

### **1. Optimize Release Timing**
   - **Friday Releases:** Fridays show a significant spike in viewership hours across all languages, especially for English content. Netflix should prioritize releasing high-profile content, such as new seasons of popular shows or blockbuster movies, on Fridays to maximize viewership.
   - **Saturday Focus for Korean Content:** Average viewership for Korean content is highest on Saturdays, suggesting Saturday as an ideal release day for Korean-language content to capture its core audience.


### **2. Leverage Seasonal Trends**
   - **Summer (June) and Winter (December) Peaks:** Viewership spikes during summer and winter holidays suggest that these are ideal periods for launching highly anticipated series or movies. Netflix should schedule major global releases and marketing campaigns during these months.
   - **Holiday Themed Content:** During December, holiday-themed movies and shows (both English and Non-English) can capitalize on the holiday season's viewing trends.


### **3. Content-Type-Specific Strategy**
   - **Shows vs. Movies:** Shows generally dominate viewership trends and follow the overall seasonal spikes. Netflix should focus on producing and releasing more high-quality shows to match audience preferences.
   - **Diversify Movie Releases:** Since movies do not follow the overall trend, Netflix can experiment with releasing movies in off-peak months (e.g., February, September) to test for underserved demand.


### **4. Language-Specific Strategies**
   - **English Content:** Maintain a steady release schedule with strategic spikes around Fridays and the holiday season to maximize its broader appeal.
   - **Korean and Non-English Content:** Invest in marketing and strategic releases during months where these languages show a higher average viewership (e.g., June and December for Korean content). Saturdays can also be targeted for niche Non-English audiences.
   - **Localized Campaigns:** Promote Non-English content heavily in regions where the language is spoken natively or has a significant viewer base. Collaborate with local influencers for better reach.


### **5. Release Cadence and Quality**
   - **Fewer, High-Quality Releases:** The analysis shows no correlation between the number of releases in a month and viewership hours. Instead of focusing on quantity, Netflix should emphasize producing high-quality content that resonates with audiences.
   - **Pre-Weekend Releases:** Given that Friday releases correlate strongly with viewership spikes, content released mid-week (e.g., Wednesday or Thursday) could help build anticipation, encouraging binge-watching over the weekend.


### **6. Data-Driven Marketing Campaigns**
   - **Dynamic Promotions:** Align marketing efforts with trends observed in specific regions and languages. For instance, focus on promoting Korean content more heavily in June and December or English shows in November and December.
   - **Tailored Recommendations:** Use insights on language and genre preferences to refine Netflix’s recommendation engine, ensuring users see content tailored to their viewing habits.


### **7. Experimentation and Analytics**
   - **Day-of-the-Week Testing:** While Friday is dominant, experimenting with alternative days like Saturday or mid-week for specific genres or regions can identify new opportunities.
   - **Track Emerging Trends:** Regularly analyze shifts in viewership behavior (e.g., growth in Non-English content consumption) to adapt the strategy dynamically.


By following these recommendations, Netflix can maximize viewership, enhance user engagement, and strengthen its position as a leader in the global streaming market.